In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import random


In [2]:
np.random.seed(0)
dataset = pd.read_csv("creditcard.csv")

In [3]:
dataset.head()
### 0 if not fraudulant

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
##### pre-processing
from sklearn.preprocessing import StandardScaler

In [5]:
classifier = StandardScaler()

In [6]:
dataset['normalizedAmount'] = classifier.fit_transform(dataset.Amount.values.reshape(-1,1))

In [7]:
dataset = dataset.drop('Amount', axis=1)
dataset = dataset.drop('Time', axis=1)

In [8]:
X = dataset.iloc[:, dataset.columns != 'Class']
y = dataset.iloc[:, dataset.columns == 'Class']

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [10]:
X_train.shape

(199364, 29)

In [11]:
X_test.shape

(85443, 29)

In [12]:
######DECISION TREES

In [13]:
from sklearn.tree import DecisionTreeClassifier
decision_tree = DecisionTreeClassifier()

In [14]:
decision_tree.fit(X_train, y_train.values)

DecisionTreeClassifier()

In [15]:
y_pred = decision_tree.predict(X_test)

In [16]:
dt_score = decision_tree.score(X_test, y_test)

In [17]:
dt_score

0.9993328885923949

In [18]:
y_pred = decision_tree.predict(X)

In [19]:
y_expected = pd.DataFrame(y)

In [20]:
cm = confusion_matrix(y_expected, y_pred.round())

In [21]:
cm

array([[284293,     22],
       [    35,    457]], dtype=int64)

In [22]:
##### ensuring that the values match up
count_1 = 0
count_0 = 0
for value in dataset.Class.values:
    if value == 1:
        count_1 += 1
    else:
        count_0 += 1

In [23]:
count_1

492

In [24]:
count_0

284315

In [25]:
### Undersampling
fraud_index = np.array(dataset[dataset.Class == 1].index)

In [26]:
number_frauds = len(fraud_index)
print(number_frauds)

492


In [27]:
normal_index = np.array(dataset[dataset.Class == 0].index)

In [28]:
number_normals = len(normal_index)
print(number_normals)

284315


In [29]:
random_normal_index = np.random.choice(normal_index, number_frauds, replace=False)

In [30]:
print(len(random_normal_index))

492


In [31]:
under_sample_index = np.concatenate([fraud_index, random_normal_index])

In [32]:
print(len(under_sample_index))

984


In [33]:
under_sample_data = dataset.iloc[under_sample_index,:]

In [34]:
X_undersample = under_sample_data.iloc[:, under_sample_data.columns != 'Class']
y_undersample = under_sample_data.iloc[:, under_sample_data.columns == 'Class']

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X_undersample, y_undersample, test_size=0.3, random_state=0)

In [36]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [37]:
model = Sequential([
    Dense(units=16,input_dim=29,activation='relu'),
    Dense(units=24,activation='relu'),
    Dropout(0.5),
    Dense(units=20,activation='relu'),
    Dense(units=24,activation='relu'),
    Dense(units=1,activation='sigmoid')
])

In [38]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                480       
_________________________________________________________________
dense_1 (Dense)              (None, 24)                408       
_________________________________________________________________
dropout (Dropout)            (None, 24)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                500       
_________________________________________________________________
dense_3 (Dense)              (None, 24)                504       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 25        
Total params: 1,917
Trainable params: 1,917
Non-trainable params: 0
______________________________________________________

In [39]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size = 14, epochs=5)

Epoch 1/5
50/50 [==============================] - 0s 1ms/step - loss: 0.5965 - accuracy: 0.5610
Epoch 2/5
50/50 [==============================] - 0s 1ms/step - loss: 0.4805 - accuracy: 0.7834
Epoch 3/5
50/50 [==============================] - 0s 1ms/step - loss: 0.3935 - accuracy: 0.8517
Epoch 4/5
50/50 [==============================] - 0s 2ms/step - loss: 0.3073 - accuracy: 0.8924
Epoch 5/5
50/50 [==============================] - 0s 1ms/step - loss: 0.2822 - accuracy: 0.9026


In [40]:
score = model.evaluate(X_test, y_test)

10/10 [==============================] - 0s 1ms/step - loss: 0.2228 - accuracy: 0.9189


In [41]:
y_pred = model.predict(X_test)
y_expected = y_test

In [42]:
cm = confusion_matrix(y_expected, y_pred.round())

In [43]:
cm

array([[145,   4],
       [ 20, 127]], dtype=int64)

In [44]:

###### SMOTE oversampling #######

In [47]:

pip install -U imbalanced-learn

In [45]:
from imblearn.over_sampling import SMOTE

In [46]:
X_resample, y_resample = SMOTE().fit_sample(X, y.values.ravel())

In [48]:
X_resample = pd.DataFrame(X_resample)
y_resample = pd.DataFrame(y_resample)

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X_resample, y_resample, test_size=0.3, random_state=0)

In [50]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [51]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size = 14, epochs=5)

Epoch 1/5
28432/28432 [==============================] - 33s 1ms/step - loss: 0.0421 - accuracy: 0.9858
Epoch 2/5
28432/28432 [==============================] - 27s 936us/step - loss: 0.0170 - accuracy: 0.9955
Epoch 3/5
28432/28432 [==============================] - 26s 924us/step - loss: 0.0140 - accuracy: 0.9966
Epoch 4/5
28432/28432 [==============================] - 26s 922us/step - loss: 0.0120 - accuracy: 0.9971
Epoch 5/5
28432/28432 [==============================] - 26s 927us/step - loss: 0.0110 - accuracy: 0.9974


In [52]:
y_pred = model.predict(X_test)
y_expected = y_test
cm1 = confusion_matrix(y_expected, y_pred.round())

In [53]:
cm1

array([[84802,   370],
       [   19, 85398]], dtype=int64)